In [9]:
#pastas colocadas e que funcionam:
#seguenotasfiscais2quinzenadedezembropnz funcionando<<
#n_f52transf_jpajabjpa (deve funcionar mas só 1 file)
#n_fs1718jaymejosedelimajunio (deve funcionar mas só 2 file)
#nfs_henrique_roberval_josrogrio funcionando<<
#nfs_jarbiana não_funcionando <<<<<<<<MAS TA LENDO>>>>>>>>
#nfsref2quinzenadezembro2023 tem que ser feito com o leitor de imagem
#ren_fs535455e56edvaldodesouzasilvajunior funcionando <<<

In [45]:
import os
import csv  
import pdfplumber
import pandas as pd

essencialmente só colocar o dir da pasta na função que ele faz um csv com a informação

In [66]:
def leitor(dir):
    import os
    import csv  
    import pdfplumber
    import pandas as pd
    
    csv_file_path = dir + '/pdf_analise.csv'
    colunas = ('chave','cnpj_prestador','cnpj_tomador','valor_servico','numero_nota','data','email_tomador','email_prestador','empresa_tomador','empresa_prestador','codigo_tributacao','IRRF,CP,CSLL')            
    with open(csv_file_path,'w',newline='') as f:
        csv_writer = csv.writer(f)
        csv_writer.writerow(colunas)
    try:   
        fileExt = r".pdf"
        files = os.listdir(dir)
        pdf_files = [arq for arq in files if arq.lower().endswith('.pdf')]
        for pdf_file in pdf_files:
            with pdfplumber.open(f"{dir}/{pdf_file}") as pdf:
                text = ""
                for page in pdf.pages:
                    text += page.extract_text()
                    a = text
                    lines = a.split('\n')
                    cda=False
                    tds=False
                    vds=False
                    nfs=False
                    ne=False
                    em=0
                    cdt=False
                    counter=0
                    p_irff = False
                    for line in lines:
                        if p_irff == True:
                            irrf = line.split(' ')[0]
                            p_irff = False
                        if cdt==True:
                            counter=counter+1
                            if counter==1:
                                codigo_tributacao=line.split(' ')[0].split('-')[0]
                                desc1=line.split(' ')[1]
                                
                            if counter==2:
                                desc2=line.split(' ')[0]
                                desc=desc1+desc2
                                cdt=False
                            cdt=cdt+1
                            
                        if ne==True:
                            if em == 0:
                                email_prestador=line.split(' ')[1]
                                empresa_tomador = line.split('-')[0]
                                em =1
                            email_tomador=line.split(' ')[1]
                            empresa_prestador = line.split(' ')[0]
                            ne=False
                        if nfs == True:
                            numero_nota= line.split(' ')[0]
                            data = line.split(' ')[1]
                            nfs=False
                        if vds == True:
                            valor_servico=line.split(' ')[0]
                            vds=False
                        if tds== True:
                            cnpj_tomador = line.split(' ')[0]
                            tds = False
                        if cda == True:
                            chave = line.replace(' ','')[0:44]
                            cda=False
                        if 'ChavedeAcessodaNFS' in line:
                            cda=True
                        
                        if 'PrestadordoServiço' in line:
                            cnpj_prestador = line.split(' ')[1]
                        if 'TOMADORDOSERVIÇO' in line:
                            tds=True
                            
                        if 'ValordoServiço' in line:
                            vds=True
                            
                        if 'NúmerodaNFS' in line:
                            nfs=True
                            
                        if 'Nome/NomeEmpresarial' in line:
                            ne=True
                            
                        if 'CódigodeTributaçãoNacional' in line:
                            cdt=True
                            
                        if 'IRRF,CP,CSLL' in line:
                            p_irff=True
                    with open(csv_file_path, mode='a', encoding= 'utf8',newline='') as resultado_pesquisa:
                        writer = csv.writer(resultado_pesquisa)
                        linhas = (chave,cnpj_prestador,cnpj_tomador,valor_servico,numero_nota,data,email_tomador,email_prestador,empresa_tomador,empresa_prestador,codigo_tributacao,irrf)
                        writer.writerow(linhas)
                    print(f'{pdf_file} lido com sucesso!')
                    df = pd.read_csv(csv_file_path)
                    df.to_excel(f'{csv_file_path}.xlsx')
    except:
        print('não foi possível ler todos os dados')
            
                #(r"C:/Users/jvict/Documents/seven/seven/seguenotasfiscais2quinzenadedezembropnz/LAZARO SEGUNDA QUINZENA DE DEZEMBRO SABADO.pdf")
            #numero do documento,série,data,situação(0-regular/2-cancelada),valor serviços,valor descontos,
            #valor contabil,base de calculo,aliquota ISS,valor ISS normal,valor irrf,valor pis,valor cofins,valor csll,
            # valor crf, valor inss


In [67]:
leitor(r"C:\Users\jvict\Documents\seven\seven\seguenotasfiscais2quinzenadedezembropnz")

BERNEVLDO SEGUND AQUINZENA DE DEZEMBRO.pdf lido com sucesso!
DANIEL SEGUNDA QUINZENA DE DEZEMBRO MIOLO.pdf lido com sucesso!
DANIEL SEGUNDA QUINZENA SEMANAL DEZEMBRO.pdf lido com sucesso!
DANIEL SEGUNDA QUINZENA SEMANAL DEZEMBRO_1.pdf lido com sucesso!
LAZARO SEGUNDA QUINZENA DE DEZEMBRO SABADO.pdf lido com sucesso!
LAZARO SEGUNDA QUINZENA DE DEZEMBRO SEMANAL.pdf lido com sucesso!
LAZRO SEGNDA QUINZENA DESEMBRO MIOLO.pdf lido com sucesso!
MARCOS JOSE COELHO 2 QUINZENA DE DEZEMBRO.pdf lido com sucesso!
MARIA SAMARA SEGUNDA QUINZENA DE DEZEMBRO.pdf lido com sucesso!
nota progresso jose machado dezembro.pdf lido com sucesso!
nota_708968 alarme janiero.pdf lido com sucesso!
nota_758460 (NFSe) alrme janeiro.pdf lido com sucesso!


In [129]:
fileExt = r".pdf"
dir = r"C:\Users\jvict\Documents\seven\seven\seguenotasfiscais2quinzenadedezembropnz"
files = os.listdir(dir)
pdf_files = [arq for arq in files if arq.lower().endswith('.pdf')]
for pdf_file in pdf_files:
    with open('extração_de_dados.csv','w') as fp:
        fp.write('cnpj_prestador,razao_social,uf_serv,endereco,n_nota,serie,data,VALOR_SERVIÇO ,DESCONTO_INCONDICIONAL,VALOR_SERVIÇO,DEDUÇÕES,VALOR_CONTÁBIL,BASE_CÁLCULO,ALIQUOTA_ISS,ISS,PIS,COFINS,CSLL,INSS\n')
    with pdfplumber.open(f'{dir}') as pdf:
                text = ""
                cnpj_prestador = 0
                naprox = False
                csll_prox = False
                ded_prox = False
                end = 0
                rs=0
                uf=False
                serie = '-'
                CRF = '-'
                for page in pdf.pages:
                    text += page.extract_text()
                    a = text
                    lines = a.split('\n')
                    for line in lines: 
                        if 'NOTA FISCAL DE SERVIÇOS ELETRÔNICA - NFSe' == lines[0]:
                            if uf==True:
                                uf_serv=line.split('-')[1]
                                uf=False
                            if ded_prox == True:
                                VALOR_SERVIÇO = line.split(' ')[0].replace(',','.')
                                DEDUÇÕES = line.split(' ')[1].replace(',','.')
                                DESCONTO_INCONDICIONAL = line.split(' ')[2].replace(',','.')
                                BASE_CÁLCULO = line.split(' ')[3].replace(',','.')
                                
                                
                                ALIQUOTA_ISS = line.split(' ')[4].replace(',','.')
                                ISS = line.split(' ')[5].replace(',','.')
                                ded_prox = False
                            if csll_prox == True:
                                INSS = line.split(' ')[0].replace(',','.')
                                IR = line.split(' ')[1].replace(',','.')
                                CSLL = line.split(' ')[2].replace(',','.')
                                COFINS = line.split(' ')[3].replace(',','.')
                                PIS = line.split(' ')[4].replace(',','.')
                                csll_prox = False
                            if naprox == True:
                                n_nota = line.split(' ')[0]
                                naprox=False
                            if 'CPF/CNPJ:' in line:
                                cnpj_tomador = line.split(' ')[-1]
                                if cnpj_prestador == 0:
                                    cnpj_prestador = line.split(' ')[-1]
                            if 'Emitido em' in line:
                                data = line.split(' ')[2] 
                            if 'Exigível Tributacao Normal' in line:
                                naprox=True
                            if 'CSLL (R$)' in line:
                                csll_prox = True
                                
                            if 'DEDUÇÕES' in line:
                                ded_prox = True
                            
                            if 'Endereço:' in line:
                                end = end+1
                                if end ==2:
                                    endereco=line.split(':')[1].replace(',','')
                                    uf=True
                                    
                            if 'Razão Social:' in line:
                                rs=rs+1
                                if rs==1:
                                    razao_social=line.split(':')[1]
                    dados = cnpj_prestador,razao_social,uf_serv,endereco,n_nota,serie,data,VALOR_SERVIÇO ,DESCONTO_INCONDICIONAL,VALOR_SERVIÇO,DEDUÇÕES,VALOR_SERVIÇO,BASE_CÁLCULO,ALIQUOTA_ISS,ISS,PIS,COFINS,CSLL,INSS
                    dados=str(dados)
                    with open('extração_de_dados.csv','a') as fp:
                        fp.write(f'{dados}\n')
                    print('lido com sucesso')
                

teste

In [130]:
ExtratordeInfo2(r"C:\Users\jvict\Documents\seven\seven\seguenotasfiscais2quinzenadedezembropnz")

nao akjskasj
nao akjskasj
nao akjskasj
nao akjskasj
('24.254.072/0001-23', ' LAZARO JOEDSON MOURA SILVA', ' PE ', ' Rua NUCLEO RURAL PSNC - N7 115 LETRA D - ZONA RURAL', '119', '-', '02/01/2024', '300.00', '0.00', '300.00', '0.00', '300.00', '300.00', '2.00', '6.00', '0.00', '0.00', '0.00', '0.00')
LAZARO SEGUNDA QUINZENA DE DEZEMBRO SABADO.pdf lido com sucesso!
nao akjskasj
('24.254.072/0001-23', ' LAZARO JOEDSON MOURA SILVA', ' PE ', ' Rua NUCLEO RURAL PSNC - N7 115 LETRA D - ZONA RURAL', '118', '-', '02/01/2024', '2.400.00', '0.00', '2.400.00', '0.00', '2.400.00', '2.400.00', '2.00', '48.00', '0.00', '0.00', '0.00', '0.00')
LAZARO SEGUNDA QUINZENA DE DEZEMBRO SEMANAL.pdf lido com sucesso!
nao akjskasj
('24.254.072/0001-23', ' LAZARO JOEDSON MOURA SILVA', ' PE ', ' Rua NUCLEO RURAL PSNC - N7 115 LETRA D - ZONA RURAL', '120', '-', '02/01/2024', '360.00', '0.00', '360.00', '0.00', '360.00', '360.00', '2.00', '7.20', '0.00', '0.00', '0.00', '0.00')
LAZRO SEGNDA QUINZENA DESEMBRO MIOLO.p

In [91]:
def ExtratordeInfo(dir):
    import os
    import csv  
    import pdfplumber
    import pandas as pd
    
    with open('extração_de_dados.csv','w') as fp:
        fp.write('cnpj_prestador,razao_social,uf_serv,endereco,n_nota,serie,data,VALOR_SERVIÇO ,DESCONTO_INCONDICIONAL,VALOR_SERVIÇO,DEDUÇÕES,VALOR_CONTÁBIL,BASE_CÁLCULO,ALIQUOTA_ISS,ISS,PIS,COFINS,CSLL,INSS\n')
    with pdfplumber.open(dir) as pdf:
                text = ""
                cnpj_prestador = 0
                naprox = False
                csll_prox = False
                ded_prox = False
                end = 0
                rs=0
                uf=False
                serie = '-'
                CRF = '-'
                for page in pdf.pages:
                    text += page.extract_text()
                    a = text
                    lines = a.split('\n')
                    for line in lines: 
                        if 'NOTA FISCAL DE SERVIÇOS ELETRÔNICA - NFSe' == lines[0]:
                            if uf==True:
                                uf_serv=line.split('-')[1]
                                uf=False
                            if ded_prox == True:
                                VALOR_SERVIÇO = line.split(' ')[0].replace(',','.')
                                DEDUÇÕES = line.split(' ')[1].replace(',','.')
                                DESCONTO_INCONDICIONAL = line.split(' ')[2].replace(',','.')
                                BASE_CÁLCULO = line.split(' ')[3].replace(',','.')
                                
                                
                                ALIQUOTA_ISS = line.split(' ')[4].replace(',','.')
                                ISS = line.split(' ')[5].replace(',','.')
                                ded_prox = False
                            if csll_prox == True:
                                INSS = line.split(' ')[0].replace(',','.')
                                IR = line.split(' ')[1].replace(',','.')
                                CSLL = line.split(' ')[2].replace(',','.')
                                COFINS = line.split(' ')[3].replace(',','.')
                                PIS = line.split(' ')[4].replace(',','.')
                                csll_prox = False
                            if naprox == True:
                                n_nota = line.split(' ')[0]
                                naprox=False
                            if 'CPF/CNPJ:' in line:
                                cnpj_tomador = line.split(' ')[-1]
                                if cnpj_prestador == 0:
                                    cnpj_prestador = line.split(' ')[-1]
                            if 'Emitido em' in line:
                                data = line.split(' ')[2] 
                            if 'Exigível Tributacao Normal' in line:
                                naprox=True
                            if 'CSLL (R$)' in line:
                                csll_prox = True
                                
                            if 'DEDUÇÕES' in line:
                                ded_prox = True
                            
                            if 'Endereço:' in line:
                                end = end+1
                                if end ==2:
                                    endereco=line.split(':')[1].replace(',','')
                                    uf=True
                                    
                            if 'Razão Social:' in line:
                                rs=rs+1
                                if rs==1:
                                    razao_social=line.split(':')[1]
                    dados = cnpj_prestador,razao_social,uf_serv,endereco,n_nota,serie,data,VALOR_SERVIÇO ,DESCONTO_INCONDICIONAL,VALOR_SERVIÇO,DEDUÇÕES,VALOR_SERVIÇO,BASE_CÁLCULO,ALIQUOTA_ISS,ISS,PIS,COFINS,CSLL,INSS
                    dados=str(dados)
                    with open('extração_de_dados.csv','a') as fp:
                        fp.write(f'{dados}\n')
                    print('lido com sucesso')
                

In [92]:
ExtratordeInfo(r"C:\Users\jvict\Documents\seven\seven\seguenotasfiscais2quinzenadedezembropnz\LAZARO SEGUNDA QUINZENA DE DEZEMBRO SABADO.pdf")

lido com sucesso


esse funciona

In [33]:
import pdfplumber
cont = 0
with pdfplumber.open(r"C:\Users\jvict\Documents\seven\seven\seguenotasfiscais2quinzenadedezembropnz\nota_708968 alarme janiero.pdf") as pdf:
            for page in pdf.pages:
                text += page.extract_text()
                a = text
                cnpj = 0
                razao = 0
                endereco=0
                uf = 0
                serie = 0
                situacao = 0
                descontos = 0
                prox = False
                prox1 = 0
                rpt = False
                lines = a.split('\n')
                for line in lines: 
                    #print(line)
                    if lines[0][0:4] == 'N° :':
                        if rpt == True:
                            COFINS = line.split("R$")[1]
                            CSLL =line.split("R$")[2]
                            INSS =line.split("R$")[3]
                            IRPJ =line.split("R$")[4]
                            PIS =line.split("R$")[5]
                            rpt = False
                        if prox == True:
                            if prox1 == 0:
                                deducoes = line.split("R$")[1]
                                Base_calculo =line.split("R$")[2]
                                aliquota =line.split("R$")[3]
                                ISS =line.split("R$")[4]
                                prox1 = prox+1
                            prox=False
                        n_nota = lines[0].split(' ')[2].strip()
                        if 'CPF/CNPJ:' in line:
                            if cnpj == 0:
                                cnpj = line.split(':')[1].strip()
                        if 'Nome/Razão Social' in line:
                            if razao == 0:
                                razao = line.split(':')[1].strip()
                        if 'Endereço:' in line:
                            if endereco == 0:
                                endereco = line.split(':')[1].split(',')[0].strip().replace(':','')
                        if 'UF:' in line:
                            if uf ==0:
                                uf = line.split(' ')[3]
                                municipio = line.split(' ')[1]
                        if 'Data emissão:' in line:
                            data = line.split(' ')[2]
                        if 'VALOR R$ ' in line:
                            VALOR_SERVIÇO = line.split(' ')[3]
                        if 'Valor Total das Deduções' in line:
                            prox = True
                        if 'COFINS CSLL INSS IRPJ PIS' in line:
                            rpt = True  
                print(cnpj,razao,uf,municipio,endereco,n_nota,serie,data,serie,VALOR_SERVIÇO ,deducoes,VALOR_SERVIÇO,Base_calculo,aliquota,ISS,PIS,COFINS,CSLL,IRPJ,INSS)
                        

16.693.500/0001-96 VFS SISTEMA ELETRONICO DE ALARME LTDA PE RECIFE RUA IRENE RAMOS GOMES DE MATTOS 708968 0 01/01/2024 0 143,00  0,00  143,00  0,00   0,00   0,00   0,00  0,00   0,00   0,00   0,00 


In [12]:
leitor(r"C:\Users\jvict\Documents\seven\seven")

nfs-e_16 Evandro Pereira.pdf lido com sucesso!
PROCEDIMENTO LANÇAMENTO VFS SISTEMAS.pdf lido com sucesso!
PROCEDIMENTO LANÇAMENTO VFS SISTEMAS.pdf lido com sucesso!
PROCEDIMENTO LANÇAMENTO VFS SISTEMAS.pdf lido com sucesso!
PROCEDIMENTO LANÇAMENTO VFS SISTEMAS.pdf lido com sucesso!
[05230009004019]LOJA87EQUATORIAL.PDF lido com sucesso!
[05230009005333]LOJA93ENERGISA.pdf lido com sucesso!
[05230009005333]LOJA93ENERGISA.pdf lido com sucesso!


In [1]:
from extrator_info import *

In [2]:
ExtratordeInfo2(r'C:\Users\jvict\Documents\seven\seven\seguenotasfiscais2quinzenadedezembropnz')

Não foi possível ler o arquivo
Não foi possível ler o arquivo
Não foi possível ler o arquivo
Não foi possível ler o arquivo
lido com sucesso
lido com sucesso
lido com sucesso
lido com sucesso
lido com sucesso
lido com sucesso
lido com sucesso
lido com sucesso


In [1]:
from extrator_info import *

In [3]:
ExtratordeInfo2(r"C:\Users\jvict\Documents\seven\seven\ren_fs535455e56edvaldodesouzasilvajunior")

Não foi possível ler o arquivo NF 53 HZD6008 COLETA FRIOVIX 18 A 29.pdf
Não foi possível ler o arquivo NF 54 HVB8D40 COLETA MOTOCICLO 18-12.pdf
Não foi possível ler o arquivo NF 55 HVB8D40 COLETA E ENTREGA 18 A 29.pdf
Não foi possível ler o arquivo NF 56 KFL5B26 CARRO EXTRA.pdf
